In [ ]:
from gcaops.graph.formality_graph import FormalityGraph
from gcaops.algebra.differential_polynomial_ring import DifferentialPolynomialRing
from gcaops.algebra.superfunction_algebra import SuperfunctionAlgebra
from gcaops.graph.undirected_graph_complex import UndirectedGraphComplex
from gcaops.graph.directed_graph_complex import DirectedGraphComplex

import itertools
import numpy as np
from multiprocessing import Pool

X_graph_encodings=[]
for i1 in [1,2,3,4,5,7,8,9,10]:
    for index_choice_1 in itertools.combinations(range(10), int(2)):
        if index_choice_1[0]==1 or index_choice_1[1]==1 or index_choice_1[0]+5==index_choice_1[1]:
            continue
        for index_choice_2 in itertools.combinations(range(10), int(2)):
            if index_choice_2[0]==2 or index_choice_2[1]==2 or index_choice_2[0]+5==index_choice_2[1]:
                continue
            for index_choice_3 in itertools.combinations(range(10), int(2)):
                if index_choice_3[0]==3 or index_choice_3[1]==3 or index_choice_3[0]+5==index_choice_3[1]:
                    continue
                for index_choice_4 in itertools.combinations(range(10), int(2)):
                    if index_choice_4[0]==4 or index_choice_4[1]==4 or index_choice_4[0]+5==index_choice_4[1]:
                        continue
                    X_graph_encodings.append((0, i1, 6, index_choice_1[0]+1, index_choice_1[1]+1, 7, index_choice_2[0]+1, index_choice_2[1]+1, 8, index_choice_3[0]+1, index_choice_3[1]+1, 9, index_choice_4[0]+1, index_choice_4[1]+1, 10))

def encoding_to_graph(encoding):
    targets = [encoding[0:3], encoding[3:6], encoding[6:9], encoding[9:12], encoding[12:15]]
    edges = sum([[(k+1,v) for v in t] for (k,t) in enumerate(targets)], [])
    return FormalityGraph(1, 10, edges)

X_graphs = []
with Pool(processes=32) as pool:
    X_graphs = list(pool.imap(encoding_to_graph, X_graph_encodings))
print('We have', len(X_graphs), 'graphs.\n')

X_graphs_iso={}
X_encodings_iso=[]
i=-1
for g in X_graphs:
    i+= 1
    h=tuple(g.canonical_form().edges())
    if not h in X_graphs_iso:
        X_graphs_iso[h]=g
        X_encodings_iso.append(X_graph_encodings[i])
X_graphs_iso=list(X_graphs_iso.values())

if len(X_graphs_iso)!= len(X_encodings_iso):
    print('There is a mistake computing the encodings corresponding to the nonisomorphic graphs.')

print ('There are', len(X_graphs_iso), 'nonisomorphic graphs.\n')

D3 = DifferentialPolynomialRing(QQ, ('rho','a'), ('x','y','z'), max_differential_orders=[5+1,5+1]) 
rho, a = D3.fibre_variables()
x,y,z = D3.base_variables()
even_coords = [x,y,z]

S3.<xi0,xi1,xi2> = SuperfunctionAlgebra(D3, D3.base_variables()) 
xi = S3.gens()
odd_coords = xi

epsilon = xi[0]*xi[1]*xi[2]
E = x*xi[0] + y*xi[1] + z*xi[2]
def evaluate_graph(g): 
    result = S3.zero()
    for index_choice in itertools.product(itertools.permutations(range(3)), repeat=5):
        sign = epsilon[index_choice[0]] * epsilon[index_choice[1]] * epsilon[index_choice[2]] * epsilon[index_choice[3]] * epsilon[index_choice[4]]
        vertex_content = [E, S3(rho), S3(rho), S3(rho), S3(rho), S3(rho), S3(a), S3(a), S3(a), S3(a), S3(a)]
        for ((source, target), index) in zip(g.edges(), sum(map(list, index_choice), [])):
            vertex_content[target] = vertex_content[target].derivative(even_coords[index])
        result += sign * prod(vertex_content)
    return result

X_vector_fields = []
with Pool(processes=32) as pool:
    X_vector_fields = list(pool.imap(evaluate_graph, X_graphs_iso))

zeros=X_vector_fields.count(0)
print('There are', zeros, 'graphs that evaluate to 0 under the morhpism from graphs to multivectors.')

X_monomial_basis = [set([]) for i in range(3)]
for i in range(3):
    for X in X_vector_fields:
        X_monomial_basis[i] |= set(X[i].monomials())
X_monomial_basis = [list(b) for b in X_monomial_basis]
X_monomial_count = sum(len(b) for b in X_monomial_basis)

X_evaluation_matrix = matrix(QQ, X_monomial_count, len(X_vector_fields))
for i in range(len(X_vector_fields)):
    v = vector(QQ, X_monomial_count)
    index_shift = 0
    for j in range(3):
        f = X_vector_fields[i][j]
        for coeff, monomial in zip(f.coefficients(), f.monomials()):
            monomial_index = X_monomial_basis[j].index(monomial)
            v[index_shift + monomial_index] = coeff
        index_shift += len(X_monomial_basis[j])
    X_evaluation_matrix.set_column(i, v)

nullity = X_evaluation_matrix.right_nullity()
print('The vector fields have', nullity, 'linear relations among themselves.\n')

pivots = X_evaluation_matrix.pivots()

lin_ind_vector_fields=[]
lin_ind_encodings=[]
for i in pivots:
    lin_ind_vector_fields.append(X_vector_fields[i])
    lin_ind_encodings.append(X_encodings_iso[i])

print('We have', len(lin_ind_vector_fields), 'linearly independent vector fields.\n')

P= (rho*epsilon).bracket(a)
if P.bracket(P)!=0:
    print('P is not a Poisson bivector. \n')

GC = UndirectedGraphComplex(QQ, implementation='vector', sparse=True)
fivewheel_cocycle = GC.cohomology_basis(6,10)[0]; fivewheel_cocycle
dGC = DirectedGraphComplex(QQ, implementation='vector')
fivewheel_oriented = dGC(fivewheel_cocycle)
fivewheel_oriented_filtered = fivewheel_oriented.filter(max_out_degree=2)
fivewheel_operation = S3.graph_operation(fivewheel_oriented_filtered)
Q_fivewheel= fivewheel_operation(P,P,P,P,P,P)

X_bivectors=[]
for X in X_lin_ind_vector_fields:
    X_bivectors.append(P.bracket(X))

zero_bivectors = X_bivectors.count(0)
print('There are', zero_bivectors, 'vector fields in lin_ind_vector_fields that evaluate to 0 bivectors after taking the Schouten bracket with P .')

Q_monomial_basis={}
from itertools import combinations
for i,j in combinations(range(2),2):
    Q_monomial_basis[i,j]=set(Q_fivewheel[i,j].monomials())
    for P_X in X_bivectors:
        Q_monomial_basis[i,j]|= set(P_X[i,j].monomials())
        
Q_monomial_basis={idx: list(b) for idx, b in Q_monomial_basis.items()}
Q_monomial_index= {idx:{m:k for k,m in enumerate(b)} for idx, b in Q_monomial_basis.items()}
Q_monomial_count=sum(len(b) for b in Q_monomial_basis.values());

Q_fivewheel_vector= vector(QQ, Q_monomial_count, sparse=True)
index_shift=0
for i,j in Q_monomial_basis:
    for coeff, monomial in Q_fivewheel[i,j]:
        monomial_index= Q_monomial_index[i,j][monomial]
        Q_fivewheel_vector[monomial_index+index_shift]=coeff
    index_shift+=len(Q_monomial_basis[i,j])

# We create the matrix that represents the X_bivectors in terms of the monomials.
X_bivector_evaluation_matrix= matrix(QQ, Q_monomial_count, len(X_bivectors), sparse=True)
for k in range(len(X_bivectors)):
    P_X=X_bivectors[k]
    v=vector(QQ,Q_monomial_count, sparse=True)
    index_shift=0
    for i,j in Q_monomial_basis:
        for coeff, monomial in P_X[i,j]:
            monomial_index=Q_monomial_index[i,j][monomial]
            v[monomial_index+index_shift]=coeff
        index_shift+=len(Q_monomial_basis[i,j])
    X_bivector_evaluation_matrix.set_column(k,v)

X_solution=X_bivector_evaluation_matrix.solve_right(Q_fivewheel_vector)

length_sol=np.nonzero(X_solution)
print('The are in total', length_sol[0].size, 'graph appearing in the solution. The following graphs appear in the solution', length_sol, 'with the coefficients \n')
for i in range(length_sol[0].size):
    print(X_solution[length_sol[0][i]])
print('The corresponding graph encodings are \n')
for i in range(length_sol[0].size):
    print(lin_ind_encodings[length_sol[0][i]])

X_cocycle_space= X_bivector_evaluation_matrix.right_kernel().quotient(X_evaluation_matrix.right_kernel())
X_cocycles=[X_cocycle_space.lift(v) for v in X_cocycle_space.basis()]
if all(X_bivector_evaluation_matrix*X_cocycle==0 for X_cocycle in X_cocycles)!= True:
    print('There is an error in the cocycle space.')
print('The space of solutions to the homogeneous system has dimension', X_cocycle_space.dimension(), )